In [1]:
import pandas as pd
import pymongo
import numpy as np
import data_mongo as dm 

In [2]:
starbucks = pd.read_csv('../DB/starbucks.csv', usecols = [0, 4, 5, 11, 12])

In [3]:
starbucks.head()

,Brand,Street Address,City,Longitude,Latitude
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,1.53,42.51
1,Starbucks,"1 Street 69, Al Jarf",Ajman,55.47,25.42
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,55.47,25.39
3,Starbucks,Al Salam Street,Abu Dhabi,54.38,24.48
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,54.54,24.51


In [4]:
starbucks = starbucks[starbucks.Longitude.apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
starbucks = starbucks[starbucks.Latitude.apply(lambda x: type(x) in [int, np.int64, float, np.float64])]

In [5]:
starbucks['Localization']=np.vectorize(dm.geopoint)(starbucks['Longitude'], starbucks['Latitude'])

In [6]:
starbucks.head()

,Brand,Street Address,City,Longitude,Latitude,Localization
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,1.53,42.51,"{'type': 'Point', 'coordinates': [1.53, 42.51]}"
1,Starbucks,"1 Street 69, Al Jarf",Ajman,55.47,25.42,"{'type': 'Point', 'coordinates': [55.47, 25.42]}"
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,55.47,25.39,"{'type': 'Point', 'coordinates': [55.47, 25.39]}"
3,Starbucks,Al Salam Street,Abu Dhabi,54.38,24.48,"{'type': 'Point', 'coordinates': [54.38, 24.48]}"
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,54.54,24.51,"{'type': 'Point', 'coordinates': [54.54, 24.51]}"


In [7]:
sbux_db, sbux_coll = dm.connect_collection('companies', 'starbucks')

In [8]:
sbux_coll.insert_many(starbucks.to_dict('record'))

In [13]:
# sbux_coll.delete_many({'$or':[{"Latitude":{'$eq':None}},{"Longitude":{'$eq':None}}]})

sbux_coll.delete_one({"Street Address":"23 & 23-1, Yoido-Dong, Yongdongpo-Gu, 1F, #101"})

In [14]:
sbux_coll.create_index ([('Localization', pymongo.GEOSPHERE)])

'Localization_2dsphere'